# Web Scraping Magyar Nemzet (without API)

## Putting search results from multiple search pages from the same site in one dataframe

In [1]:
# Libraries

from requests_html import HTMLSession
import pandas as pd

pd.set_option("display.max_columns",101)

/opt/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# Collecting links to all articles based on keyword and search page number from Magyar Nemzet, a state-owned Hungarian newspaper

def news_articles_from_given_page (keyword, page_number, fromdate='2010-01-01', todate='2021-05-05'):
    '''Collects the links of the search results from a given page, to a given term within a given time interval from Magyar Nemzet. 'fromdate' and 'todate' are in format YYYY-MM-DD'''
    mn_url = 'https://magyarnemzet.hu/page/' + str(page_number) + '/?after=' + fromdate + '&author&before=' + todate + '&category=0&s=' + keyword
    print(mn_url)
    session = HTMLSession()
    r = session.get(mn_url)
    articles_url =[] # empty list to collectthe links into
    for i in r.html.find('h2'):
        for link in i.links:
            articles_url.append(link)
    return articles_url
    

In [3]:
# Test

news_articles_from_given_page('Európai Unió',1)

https://magyarnemzet.hu/page/1/?after=2010-01-01&author&before=2021-05-05&category=0&s=Európai Unió


['https://magyarnemzet.hu/belfold/orban-viktor-ma-nem-letezik-liberalis-demokracia-csak-liberalis-nem-demokracia-9758114/',
 'https://magyarnemzet.hu/gazdasag/uj-elelmiszerkent-forgalmazhato-a-lisztkukac-az-europai-unioban-9755897/',
 'https://magyarnemzet.hu/gazdasag/nagy-marton-aprilisban-mar-sok-szektor-a-valsag-elotti-szinteken-van-9757079/',
 'https://magyarnemzet.hu/belfold/irdala-hu-belgiumban-karnyujtasnyi-kozelsegbe-kerult-a-siker-9754478/',
 'https://magyarnemzet.hu/digitalia/mesterseges-intelligencia-segiti-a-csillagok-jobb-megismereset-9753980/',
 'https://magyarnemzet.hu/kulfold/veszelybe-kerult-a-kinaval-kotott-unios-befektetesi-egyezmeny-9753644/',
 'https://magyarnemzet.hu/belfold/baloldali-politikai-aktivista-kezeben-a-magyar-jogallamisagi-jelentes-9754337/',
 'https://magyarnemzet.hu/kulfold/korrupcios-ugyekbe-keveredhetett-az-europai-unio-egeszsegugyi-biztosa-9752951/',
 'https://magyarnemzet.hu/gazdasag/megsem-vesz-hibrid-motorvonatokat-a-mav-start-9749255/',
 'http

In [4]:
# Putting news articles from multiple search pages in a list using the previously defined function

article_links=[]

for page_number in range(1, 40):
    new_links = news_articles_from_given_page('Európai Unió', page_number)
    article_links = article_links + new_links

https://magyarnemzet.hu/page/1/?after=2010-01-01&author&before=2021-05-05&category=0&s=Európai Unió
https://magyarnemzet.hu/page/2/?after=2010-01-01&author&before=2021-05-05&category=0&s=Európai Unió
https://magyarnemzet.hu/page/3/?after=2010-01-01&author&before=2021-05-05&category=0&s=Európai Unió
https://magyarnemzet.hu/page/4/?after=2010-01-01&author&before=2021-05-05&category=0&s=Európai Unió
https://magyarnemzet.hu/page/5/?after=2010-01-01&author&before=2021-05-05&category=0&s=Európai Unió
https://magyarnemzet.hu/page/6/?after=2010-01-01&author&before=2021-05-05&category=0&s=Európai Unió
https://magyarnemzet.hu/page/7/?after=2010-01-01&author&before=2021-05-05&category=0&s=Európai Unió
https://magyarnemzet.hu/page/8/?after=2010-01-01&author&before=2021-05-05&category=0&s=Európai Unió
https://magyarnemzet.hu/page/9/?after=2010-01-01&author&before=2021-05-05&category=0&s=Európai Unió
https://magyarnemzet.hu/page/10/?after=2010-01-01&author&before=2021-05-05&category=0&s=Európai Unió

In [5]:
# Function that scrapes the first search result (one article among the many search results) from a single search page -> article title, author, date, text... (put in a dictionary, so we can later comply a json file.)
# so it opens the first find -> article_links[0]

def scrape_magyar_nemzet(url):
    session = HTMLSession()
    page=session.get(url)
    html = page.html

    title = html.find('h1')[0].text
    if html.find('.en-article-subtitle')==[]:
        subtitle = 'N/A'
    else:
        subtitle = html.find('.en-article-subtitle')[0].text
    if html.find('.en-article-source')==[]:
        source = 'N/A'
    else:
        source = html.find('.en-article-source')[0].text
    if html.find('.en-article-author')==[]:
        author = 'N/A'
    else:
        author = html.find('.en-article-author')[0].text
    date = html.find('.en-article-dates-main')[0].text
   
    text = '' 
    
    for item in html.find('p'):
        text = text + ' ' + item.text
    
    d = {'title' : title,
         'subtitle' : subtitle,
         'author' : author,
         'source' : source,
         'date' : date,
         'text' : text}
    
    return d

In [6]:
article_links[0]

'https://magyarnemzet.hu/belfold/orban-viktor-ma-nem-letezik-liberalis-demokracia-csak-liberalis-nem-demokracia-9758114/'

In [7]:
scrape_magyar_nemzet(article_links[0])

{'title': 'Orbán Viktor: Meg akarjuk változtatni Brüsszelt',
 'subtitle': 'N/A',
 'author': 'N/A',
 'source': 'MTI',
 'date': '2021. május 5. szerda 22:41',
 'text': ' A kevés szlovák konzervatív irányultságú hírportál egyikén megjelent, tematikailag átfogó nagyinterjúban a magyar miniszterelnök időszerű és nagyobb időtávlatokat átölelő témákról egyaránt nyilatkozott, köztük az elmúlt évtizedek történéseiről, az Európai Unió és Közép-Európa jövőjéről, a migrációról és a vakcinapolitikáról is. A miniszterelnök egy az „illiberális országépítés” kapcsán neki feltett kérdésre válaszolva azt mondta: ma nem létezik liberális demokrácia, csak „liberális nem-demokrácia”, mivel van benne liberalizmus, de nincs benne demokrácia. A miniszterelnök hangsúlyozta: a liberálisok véleményhegemóniára törekednek, és erre szolgál a politikai korrektség, aminek segítségével megbélyegzik a konzervatívokat és kereszténydemokratákat, és igyekeznek ellehetetleníteni őket. „Én harcolok a liberálisokkal szemben 

In [8]:
# Loop to scrape all (a list of X) articles from multiple pages of the actual search results

articles =[]

for url in article_links:
    if 'magyarnemzet' in url:
        print(url)
        details = scrape_magyar_nemzet(url)
        details['url'] = url
        articles.append(details)

https://magyarnemzet.hu/belfold/orban-viktor-ma-nem-letezik-liberalis-demokracia-csak-liberalis-nem-demokracia-9758114/
https://magyarnemzet.hu/gazdasag/uj-elelmiszerkent-forgalmazhato-a-lisztkukac-az-europai-unioban-9755897/
https://magyarnemzet.hu/gazdasag/nagy-marton-aprilisban-mar-sok-szektor-a-valsag-elotti-szinteken-van-9757079/
https://magyarnemzet.hu/belfold/irdala-hu-belgiumban-karnyujtasnyi-kozelsegbe-kerult-a-siker-9754478/
https://magyarnemzet.hu/digitalia/mesterseges-intelligencia-segiti-a-csillagok-jobb-megismereset-9753980/
https://magyarnemzet.hu/kulfold/veszelybe-kerult-a-kinaval-kotott-unios-befektetesi-egyezmeny-9753644/
https://magyarnemzet.hu/belfold/baloldali-politikai-aktivista-kezeben-a-magyar-jogallamisagi-jelentes-9754337/
https://magyarnemzet.hu/kulfold/korrupcios-ugyekbe-keveredhetett-az-europai-unio-egeszsegugyi-biztosa-9752951/
https://magyarnemzet.hu/gazdasag/megsem-vesz-hibrid-motorvonatokat-a-mav-start-9749255/
https://magyarnemzet.hu/belfold/atlathatob

https://magyarnemzet.hu/orszagszerte/sertespestis-harmadara-csokkent-nograd-megye-vaddisznoallomanya-9716963/
https://magyarnemzet.hu/kulfold/megvesztegethettek-a-vakcinagyartok-az-eu-biztost-9717707/
https://magyarnemzet.hu/belfold/hollik-istvan-a-baloldalnak-a-sajat-valasztoik-eletet-veszelyeztetik-szegyeljek-magugat-9716951/
https://magyarnemzet.hu/belfold/schanda-tamas-a-baloldal-a-magyar-fiatalok-ellen-beszel-9717110/
https://magyarnemzet.hu/kulfold/a-cseh-oltasi-folyamatot-foleg-a-vakcinahiany-hatraltatja-9717050/
https://magyarnemzet.hu/gazdasag/megmentik-az-europai-vasuttarsasagokat-9712571/
https://magyarnemzet.hu/kulfold/kijatsszak-a-rendszert-a-migransok-9715859/
https://magyarnemzet.hu/belfold/a-honvedseg-maga-mogott-hagyta-a-baloldal-szornyu-orokseget-9715439/
https://magyarnemzet.hu/ahelyzet/gyurcsany-atvaltozasa-9716363/
https://magyarnemzet.hu/kulfold/mar-europaban-van-az-indiai-mutacio-9715778/
https://magyarnemzet.hu/gazdasag/regioelso-lesz-a-magyar-gazdasag-londoni-e

https://magyarnemzet.hu/belfold/nemzetkozi-hadgyakorlat-kezdodott-magyarorszagon-9685961/
https://magyarnemzet.hu/kulfold/651-ezer-adag-vakcinaval-segiti-az-eu-a-nyugat-balkant-9685736/
https://magyarnemzet.hu/belfold/az-oshonos-nemzeti-kisebbsegek-europa-alapveto-ertekeit-jelentik-9684977/
https://magyarnemzet.hu/belfold/a-pandemia-ellenere-megelenkultek-a-csempeszek-9684599/
https://magyarnemzet.hu/kulfold/a-v4-ek-es-az-europai-unio-is-kiall-csehorszag-mellett-9684431/
https://magyarnemzet.hu/gazdasag/a-fagyok-miatt-varhatoan-az-iden-is-alig-terem-hazai-barack-9683672/
https://magyarnemzet.hu/kulfold/eltorolnek-a-bolgar-jobboldalt-9679085/
https://magyarnemzet.hu/gazdasag/szaznyolcvan-kilometernyi-tehervasutpalya-epulhet-9680306/
https://magyarnemzet.hu/kulfold/pandora-szelenceje-nyilna-meg-a-balkani-hatarok-atrajzolasaval-9682673/
https://magyarnemzet.hu/belfold/kepes-a-kozmegelegedest-szolgalni-a-kereszteny-alapokra-epito-politika-9679208/
https://magyarnemzet.hu/ahelyzet/globalis-

https://magyarnemzet.hu/belfold/szentkiralyi-minden-negyedik-ember-megkapta-az-oltast-9645671/
https://magyarnemzet.hu/kulfold/izrael-sulyos-karokat-okozhatott-az-irani-atomletesitmenyben-9645908/
https://magyarnemzet.hu/belfold/ismet-a-kinai-vakcinat-tamadja-az-oltasellenes-baloldal-9644993/
https://magyarnemzet.hu/kulfold/salvini-a-baloldal-az-ugyeszsegeken-keresztul-tamad-9645500/
https://magyarnemzet.hu/lugas-rovat/nyitott-tarnak-9645503/
https://magyarnemzet.hu/belfold/latorcai-janos-a-brusszeli-elit-birodalmi-torekveseire-figyelmeztetett-9645083/
https://magyarnemzet.hu/kulfold/geopolitikai-haborunak-nevezte-a-vakcinabeszerzest-az-olasz-politikus-9645251/
https://magyarnemzet.hu/gazdasag/reggeli-orszagos-edukacios-program-a-fogyasztokert-9642137/
https://magyarnemzet.hu/velemeny/kirekeszto-globalista-liberalisok-9643760/
https://magyarnemzet.hu/belfold/kozonseges-szemettelepet-csinalt-az-fkf-a-sajat-letesitmenyebol-9642995/
https://magyarnemzet.hu/gazdasag/vilagi-oszkar-interju-n

https://magyarnemzet.hu/belfold/tomegek-tiltakoznak-a-genderideologia-terjesztese-ellen-9615878/
https://magyarnemzet.hu/kulfold/az-eu-a-kulfoldi-csapatok-azonnali-kivonasat-surgeti-libiabol-9614060/
https://magyarnemzet.hu/belfold/alhir-a-terezvarosi-szakrendelo-szombati-bezarasa-9613223/
https://magyarnemzet.hu/kulfold/josep-borrell-aggodik-az-orosz-csapatmozgasok-miatt-9613001/
https://magyarnemzet.hu/eletstilus/hadat-uzentek-a-szakmai-szervezetek-a-megteveszto-etelneveknek-9602978/
https://magyarnemzet.hu/lugas-rovat/haztaji-9597689/
https://magyarnemzet.hu/kulfold/tiz-ev-utan-felcsillant-a-remeny-a-bekere-libiaban-9583481/
https://magyarnemzet.hu/kulfold/az-eu-udvozolte-az-amerikai-szankciok-visszavonasat-9609503/
https://magyarnemzet.hu/sztarvilag/szikora-robert-a-jokedvkeszito-kisiparos-9608900/
https://magyarnemzet.hu/kulfold/iden-is-szuk-csaladi-korokben-unnepelnek-europa-szerte-9608528/
https://magyarnemzet.hu/gazdasag/igy-fektetjuk-be-az-unios-penzeket-9607844/
https://magya

https://magyarnemzet.hu/gazdasag/tobb-tizezer-forinttal-emelkedik-januartol-a-fiatalok-fizetese-9577139/
https://magyarnemzet.hu/velemeny/szabo-timea-ujabb-hazugsagai-9577010/
https://magyarnemzet.hu/belfold/a-baloldal-a-pandemia-alatt-6-9576194/
https://magyarnemzet.hu/kulfold/fellangol-e-az-eszakirek-haragja-a-brexit-nyoman-9574142/
https://magyarnemzet.hu/kulfold/magyarorszag-jovateszi-a-hibas-brusszeli-donteseket-9575789/
https://magyarnemzet.hu/belfold/unios-vakcinabeszerzes-kocsis-mate-a-facebookon-uzent-dobrevnek-9575309/
https://magyarnemzet.hu/kulfold/erosodik-a-nyomas-brusszelen-az-orosz-oltoanyag-beszerzesere-9573806/
https://magyarnemzet.hu/kulfold/gyorsabb-vakcinagyartast-es-szetosztast-igernek-brusszelben-9572561/
https://magyarnemzet.hu/belfold/veget-kell-vetni-a-politikai-boszorkanyuldozesnek-9572615/
https://magyarnemzet.hu/belfold/nacsa-lorinc-nagy-segitseg-a-fiataloknak-az-adomentesseg-9568739/
https://magyarnemzet.hu/velemeny/magyarorszag-a-sapka-nelkuli-nyul-957099

https://magyarnemzet.hu/belfold/vakcinahaboru-is-kialakulhat-a-vedettsegi-igazolasok-miatt-9545465/
https://magyarnemzet.hu/belfold/a-szerb-taxisok-egy-resze-az-illegalis-bevandorlokbol-el-9545255/
https://magyarnemzet.hu/gazdasag/erkezhet-a-penz-a-nyugati-kelenfold-osszekottetesre-9545447/
https://magyarnemzet.hu/gazdasag/hatpontos-segitseg-a-jarvany-miatt-bajba-jutottaknak-9545309/
https://magyarnemzet.hu/belfold/domotor-csaba-a-baloldal-barmilyen-sotet-huzasra-kepes-9545324/
https://magyarnemzet.hu/belfold/ez-tortenne-akkor-ha-csak-brusszeli-vakcinaval-oltanank-video-9543974/
https://magyarnemzet.hu/kulfold/hazudott-brusszel-a-vakcina-szerzodesekrol-ime-a-dokumentumok-9543782/
https://magyarnemzet.hu/belfold/jourova-elvtars-ez-az-itelet-9543524/
https://magyarnemzet.hu/gazdasag/unios-elvonalban-a-magyar-adorendszer-9542663/
https://magyarnemzet.hu/poszt-trauma/teljesseggel-alkalmatlan-unio-9543002/
https://magyarnemzet.hu/unilife/europai-lepteku-fejlesztesek-a-szegedi-tudomanyegyete

https://magyarnemzet.hu/belfold/a-magyarok-zavarjak-egyes-nyugati-politikusok-koreit-9512060/
https://magyarnemzet.hu/digitalia/megveheti-a-microsoft-a-jatekfejleszto-zenimaxot-9485042/
https://magyarnemzet.hu/gazdasag/szentkiralyi-alexandra-szamunkra-a-termeszetvedelem-nemzeti-ugy-9510677/
https://magyarnemzet.hu/gazdasag/egyre-tobb-orszag-venne-reszt-a-vakcinagyartasban-9508979/
https://magyarnemzet.hu/kulfold/rendkivuli-unios-csucs-johet-a-vakcinak-miatt-9510494/
https://magyarnemzet.hu/gazdasag/akar-naponta-lekerdezheto-a-forgalmi-ado-egyenlege-9509780/
https://magyarnemzet.hu/belfold/ismet-megcafolta-a-a-valosag-karacsony-gergelyt-9509534/
https://magyarnemzet.hu/velemeny/a-ket-globalis-valsag-tanulsagai-9508862/
https://magyarnemzet.hu/orszagszerte/surgeto-kerdes-ajkan-a-vizelvezeto-rendszer-atalakitasa-9485258/
https://magyarnemzet.hu/velemeny/meg-egy-mondat-a-jogallamisagrol-9505061/
https://magyarnemzet.hu/kulfold/megszabja-az-ep-a-kepviseloknek-hogy-milyen-szavakat-hasznalhat

https://magyarnemzet.hu/velemeny/bajor-bajok-9485429/
https://magyarnemzet.hu/belfold/hollik-istvan-mar-egymillioan-kaptak-meg-az-oltast-magyarorszagon-9484172/
https://magyarnemzet.hu/belfold/orban-viktor-messze-az-europai-unios-atlag-folott-vagyunk-9483197/
https://magyarnemzet.hu/kulfold/angliaban-az-iskolak-nemetorszagban-bizonyos-uzletek-nyitnak-ki-9481697/
https://magyarnemzet.hu/belfold/lengyel-ep-kepviselo-a-magyar-nemzetnek-orban-terveben-rejlik-a-jovonk-9481742/
https://magyarnemzet.hu/orszagszerte/a-sopron-balett-muveszeti-vezetoje-szamara-barmelyik-vakcina-megfelelo-9481805/
https://magyarnemzet.hu/belfold/atleptuk-az-egymillios-hatart-9481433/
https://magyarnemzet.hu/velemeny/ellenallo-a-magyar-gazdasag-9480449/
https://magyarnemzet.hu/belfold/egymillio-ember-mar-megkapta-az-oltast-magyarorszagona-9481298/
https://magyarnemzet.hu/mindmegette/mi-az-idealis-homerseklet-a-hutoszekrenyben-9481115/
https://magyarnemzet.hu/gazdasag/tartja-kivalo-pozicioit-a-budapesti-rezsi-94809

In [9]:
df = pd.DataFrame(articles)
df.sample(5)

,title,subtitle,author,source,date,text,url
118,Első amerikai elnökként beszélt örmény népirtá...,N/A,Magyar Nemzet,"MTI, BBC, Fox News",2021. április 24. szombat 20:50,"Joe Biden az első amerikai elnök, aki hivatal...",https://magyarnemzet.hu/kulfold/elso-amerikai-...
296,Novák Katalin: A valódi jobboldal legerősebb e...,N/A,N/A,PESTISRÁCOK.HU,2021. április 9. péntek 21:54,A valódi jobboldali pártok legerősebb európai...,https://magyarnemzet.hu/belfold/novak-katalin-...
549,A pusztító járvány legyőzése után a piacokért ...,N/A,Szajlai Csaba,N/A,2021. március 20. szombat 09:00,Miközben hatalmas károkat okoznak világszinte...,https://magyarnemzet.hu/velemeny/a-pusztito-ja...
415,"A húsvét miatt tormákat vizsgált a hatóság, it...",N/A,N/A,origo.hu,2021. március 30. kedd 11:40,Lengyel és magyar tormákat ellenőrzött a Nébi...,https://magyarnemzet.hu/mozaik/a-husvet-miatt-...
728,Uniós szerződés bizonyítja: Gyurcsány ismét ha...,N/A,Kreft-Horváth Márk,Facebook,2021. március 7. vasárnap 17:03,"A kormány ügyeskedik a vakcinákkal, és nem hí...",https://magyarnemzet.hu/belfold/unios-szerzode...


In [10]:
df.to_json('magyarnemzet_EU.json')

In [11]:
def has_EU(string):
    if 'Európai Unió'in string:
        return True
    if 'európai unió' in string:
        return True
    if 'EU' in string:
        return True
    if 'európai uniós' in string:
        return True
    if 'unió' in string:
        return True
    else:
        return False

In [12]:
df['EU'] = df['text'].apply(has_EU)

In [13]:
df['EU'].sum()

737

In [14]:
len(df)

780

In [15]:
df.to_csv(r'/Users/lilivandermeersch/CAB/Class Session 2 - 29:04:2021/output/Magyar_Nemzet_EU.csv', index = False)